In [1]:
import sys
sys.path.append('../../')

In [98]:
from run_bart_sum import BartSystem
from pathlib import Path
import json
from transformers import BartTokenizer
import spacy
import tqdm
import re

In [3]:
device_ = 0
if device_ > -1:
    device = f'cuda:{device_}'
else:
    device = 'cpu'
model = BartSystem.load_from_checkpoint('../../.models/bart9/epoch=1.ckpt', map_location='cuda:0')
model.model.to('cuda:0')

2020-04-12 12:20:20,771 - INFO - transformers.configuration_utils - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-cnn/config.json from cache at /home/dhruveshpate/.cache/torch/transformers/5f0de1d2bbb8eb1a3b69656622293b3328b06b701663a9d4109359751cb4e739.a42ba2fc4aa449981f74aff39e9b70a82a10ed5f615e41f7fb847905641ecc57
2020-04-12 12:20:20,775 - INFO - transformers.configuration_utils - Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": null,
  "attention_dropout": 0.0,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): SelfAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=Tr

In [4]:
tokenizer = BartTokenizer.from_pretrained('bart-large-cnn')

2020-04-12 12:20:46,403 - INFO - transformers.tokenization_utils - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-vocab.json from cache at /home/dhruveshpate/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
2020-04-12 12:20:46,405 - INFO - transformers.tokenization_utils - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-merges.txt from cache at /home/dhruveshpate/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda


In [5]:
spacy_nlp =spacy.load("en_core_web_sm")

In [6]:
# read input data
set_ = 'dev'
input_file = Path(f'../../.data/RACE/set3/{set_}.json')

In [7]:
with open(input_file) as f:
    samples = json.load(f)

In [115]:
def preprocess(q, o):
    if '_' in q: #FITB
        h = q.replace('_', o)
    else:
        h = q + ' ' + o
    return h
        

In [140]:
dots = re.compile(r"\.([\.\'\" ]{2,}[\w ]*)")
def postprocessor(inp):
    dots_removed = dots.sub('.', inp) # leave the first group as it has single period required to end the sentence
    first_sent = (list(spacy_nlp(dots_removed).sents)[0]).text.strip()
    return first_sent

In [141]:
def process_batch(batch, model, post_processor):
    # preprocess
    inp = [preprocess(ex['question'], ex['option']) for ex in batch]
    inp_tensor = tokenizer.batch_encode_plus(inp, pad_to_max_length=True, max_length=40, return_tensors='pt')
    out_tensor = model.model.generate(inp_tensor["input_ids"].cuda(),
            attention_mask=inp_tensor["attention_mask"].cuda(),
            num_beams=1, do_sample=False,no_repeat_ngram_size=2, top_k=2,
            max_length=40,
            early_stopping=True,).cpu()
    preds = [
            tokenizer.decode(
                g, skip_special_tokens=True, clean_up_tokenization_spaces=True)

            for g in out_tensor
        ]
    # post process
    for pred,ex in zip(preds, batch):
        ex['neural_hypothesis'] = postprocessor(pred)

In [142]:
batch_size = 128
def batches(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [143]:
for batch in tqdm.tqdm(batches(samples, batch_size), total=len(samples)//batch_size + 1):
    process_batch(batch, model, spacy_nlp)

  1%|▏         | 1/79 [00:05<07:21,  5.66s/it]


KeyboardInterrupt: 

In [ ]:
# write output
#output_file = input_file.absolute().parent /(f'{set_}' + '_neural.json')
#with open(output_file, 'w') as f:
#    json.dump(samples, f)

In [146]:
sublist = samples[20:30]
print('---Question+option')
for s in sublist:
    print(f"{s['question'] +' '+s['option']}")
print('---Rule based---')
for s in sublist:
    print(f"{s['hypothesis']}")
print('---Neural (BART) based---')
for s in sublist:
    print(f"{s['neural_hypothesis']}")

---Question+option
What happened to Li Ming on July 6? He crushed his forefinger while working.
What's the best title of this passage(  )? Color
What Ms. Gao and Ms. Tang said suggests that   _  . they still trust major supermarkets when shopping
How did the government deal with the disease? The government had more animals killed.
Where does Yangqi Lake locate? In Fangshan
How much should a child pay to go into Peace Park? Ten yuan.
What present did the man send to the optimist? A pair of shows
What did Mr. Brown write on the blackboard? WHICH ONE DID IT
What were the four bikers doing at the traffic lights? Selling helmets to the passersby.
Which of the following is Not mentioned in the passage ? Ordinary people travel to the twin towns .
---Rule based---
Li he crushed his forefinger while working Ming on July 6.
The best title of this passage ()?'s Color
They still trust major supermarkets when shopping said suggests that _.
The government dealt with the disease by the government had